In [1]:
# load libraries (pandas, numpy)
import pandas as pd
import numpy as np
from itertools import chain

/var/folders/t4/yngm0l7s3lb4wk6mfrj9t87c0000gn/T/ipykernel_39898/3748287199.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Loading the main dataset

In [2]:
# load the big dataset
WorkingDataSet11 = pd.read_csv('WorkingDataSet11.csv').drop(columns=['Unnamed: 0'])

In [ ]:
# Search and delete dublicates

# Check for duplicate rows
duplicate_rows = WorkingDataSet11[WorkingDataSet11.duplicated()]

# Print the duplicate rows
#print("Duplicate rows:")
#print(duplicate_rows)
#checked no dublicates

# Check for missing values in the most important columns
# Count None or NaN values in each column
missing_values_count = WorkingDataSet11.isna().sum()

# Print the count of missing values in each column
# print("Number of None or NaN values in each column:")
# print(missing_values_count)

# # Filter the DataFrame to include rows where column "A" contains NaN values
# nan_values_in_A = WorkingDataSet11[WorkingDataSet11['titles'].isna()]

# # Print the rows with NaN values in column "titles"
# print("Rows with NaN values in column 'titles':")
# print(nan_values_in_A)

# Deleting the two rows with NaN values
# Remove rows with NaN values in column "A"
WorkingDataSet11 = WorkingDataSet11.dropna(subset=['titles'])


### Create a smaller dataset

In [ ]:
# Create a subset of clincal trials between STARTING between 2000-2022
#WorkingDataSet11.columns
# Create a subset where dates are between 2000 and 2022
DataFrom2000Till2022 = WorkingDataSet11[(WorkingDataSet11['StudyStartDate'] >= '2000-01-01') & (WorkingDataSet11['StudyStartDate'] <= '2022-12-31')]

# Exclude non-relevant phases
# Unique phases 
# unique_values_A = DataFrom2000Till2022['phase'].unique()
# print(unique_values_A)
phasesCleandDF = DataFrom2000Till2022[~DataFrom2000Till2022['phase'].isin(['Not Applicable', 'Empty', 'Phase 4'])]
# Focusing the dataset on Phases I-III
phasesCleandDF['phase'] = phasesCleandDF['phase'].replace('Early Phase 1', 'Phase 1')
phasesCleandDF['phase'] = phasesCleandDF['phase'].replace('Phase 1\nPhase 2', 'Phase 2')
phasesCleandDF['phase'] = phasesCleandDF['phase'].replace('Phase 2\nPhase 3', 'Phase 3')
# unique_values_A = phasesCleandDF['phase'].unique()
# print(unique_values_A)

### load DurgBank drug names and synonyms

In [375]:
# load the Drugbank (short) DF
# merged_df = pd.read_csv('drugBankTotal.csv')
df = pd.read_csv('drugBankTotalCleaned.csv')

# # Create a new DataFrame with two columns: 'ID' and joined string value from 'A', 'B', and 'C'
name_df = pd.DataFrame({'DrugBank_ID': df['DrugBank ID'],
                       'Joined_Names': df[['Common name', 'Synonyms', 'External_IDs',"Brand_Names"]].apply(lambda x: ' | '.join(filter(None, x.dropna())), axis=1)}) 

In [307]:
name_df

,DrugBank_ID,Joined_Names
0,DB00001,"Lepirudin | [Leu1, Thr2]-63-desulfohirudin | D..."
1,DB00002,Cetuximab | Cetuximab | Cétuximab | Cetuximabu...
2,DB00003,Dornase alfa | Deoxyribonuclease (human clone ...
3,DB00004,Denileukin diftitox | Denileukin | Denileukin ...
4,DB00005,Etanercept | Etanercept | etanercept-szzs | et...
...,...,...
16570,DB18707,AVL-3288 | (z)-3-(4-chloroanilino)-n-(4-chloro...
16571,DB18708,"Alogabat | 3-pyridazinecarboxamide, 6-((5-meth..."
16572,DB18709,"Ropsacitinib | Cyclobutaneacetonitrile, 3-cyan..."
16573,DB18710,ABBV-706 | ABBV 706 | ABBV-706 | ABBV706


In [308]:
name_list_df = name_df["Joined_Names"].str.split('|')
print(name_list_df)

0        [Lepirudin ,  [Leu1, Thr2]-63-desulfohirudin ,...
1        [Cetuximab ,  Cetuximab ,  Cétuximab ,  Cetuxi...
2        [Dornase alfa ,  Deoxyribonuclease (human clon...
3        [Denileukin diftitox ,  Denileukin ,  Denileuk...
4        [Etanercept ,  Etanercept ,  etanercept-szzs ,...
                               ...                        
16570    [AVL-3288 ,  (z)-3-(4-chloroanilino)-n-(4-chlo...
16571    [Alogabat ,  3-pyridazinecarboxamide, 6-((5-me...
16572    [Ropsacitinib ,  Cyclobutaneacetonitrile, 3-cy...
16573        [ABBV-706 ,  ABBV 706 ,  ABBV-706 ,  ABBV706]
16574    [taletrectinib ,  Taletrectinib adipate ,  DS-...
Name: Joined_Names, Length: 16575, dtype: object


In [309]:
names_list_df = pd.DataFrame({'DrugBank_ID': name_df['DrugBank_ID'], 'Joined_Names': name_list_df})

### check for Synonyms per trial

In [310]:
%%time
# Create an empty list to store the names
names = []
phasesCleandDF["DrugBank_ID"] = ''

# Iterate over the rows of the Main DataFrame
for index, row in phasesCleandDF.iterrows():
    # Check if any nickname is present in the Last Name or First Name
    # Include a check for short names
    found_name = names_list_df.loc[names_list_df['Joined_Names'].apply(lambda x: any((f' {nickname.lower()} ' if len(nickname) < 4 else nickname.lower()) in row['titles'].lower() or (f' {nickname.lower()} ' if len(nickname.lower()) < 4 else nickname.lower()) in row['treatment'].lower() for nickname in x)), 'DrugBank_ID'].values.tolist()
    # If found, add the corresponding Name to the list
    if found_name:
        names.append(found_name[0])
        #phasesCleandDF.at[index, 'DrugBank_ID'] += found_name[0]
        #phasesCleandDF.loc[index, 'DrugBank_ID'] += found_name[0]
        phasesCleandDF.loc[index, 'DrugBank_ID'] += ', '.join(found_name)
    else:
        names.append(None)
        # phasesCleandDF.at[index, 'DrugBank_ID'] = None
        phasesCleandDF.loc[index, 'DrugBank_ID'] = None

# # Add the new column "Name" to the Main DataFrame
#phasesCleandDF['DrugBank_ID'] = names


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 11h 38min 54s, sys: 35.9 s, total: 11h 39min 30s
Wall time: 11h 40min 51s


In [311]:
phasesCleandDF

,Unnamed: 0,titles,Responsible_Party,sponsor,RecruitmentStatus,FirstPosted,LastUpdatePosted,condition,treatment,phase,...,FirstPosted2,LastUpdatePosted2,LastVerified,FDA,Keywords,AdditonalRelevantPoints,Post_updated_for,category,participants,DrugBank_ID
4,4,Leukocyte and Platelet-rich Fibrin (L-PRF) Eff...,"Carolina Encalada Abad, DDS, Universidad Compl...",Universidad Complutense de Madrid,"Active,notrecruiting",2022-12-22,2022-12-22,Alveolar Ridge Preservation\nDental Implants,Procedure: Tooth extraction\nProcedure: Alveol...,Phase 3,...,2022-12-22,2022-12-22,2022-12-01,Empty,['platelet-rich fibrin\r\n \t ...,Empty,0 days,Procedure,28.0,DB16604
9,9,A Phase I/II Study of WJ13404 Monotherapy in P...,"Suzhou Junjing BioSciences Co., Ltd.","Suzhou Junjing BioSciences Co., Ltd.",Recruiting,2022-12-22,2022-12-22,Locally Advanced or Metastatic NSCLC,Drug: WJ13404 tablets,Phase 2,...,2022-12-22,2022-12-22,2022-10-01,"['Studies a U.S. FDA-regulated Drug Product:',...",Empty,"['Carcinoma, Non-Small-Cell Lung ...",0 days,Drug,162.0,None
40,40,A Phase 3 Study of Obexelimab in Patients With...,"Zenas BioPharma (USA), LLC","Zenas BioPharma (USA), LLC",Recruiting,2022-12-22,2022-12-22,IgG4 Related Disease,Drug: Obexelimab\nOther: Placebo,Phase 3,...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",['IgG4\r\n \t \t IgG4...,['Immunoglobulin G4-Related Disease ...,0 days,Drug,200.0,DB15032
42,42,A Single and Multiple Ascending Dose Study of ...,Cytokinetics,Cytokinetics,Recruiting,2022-12-22,2022-12-22,Healthy Participants,Drug: CK-3828136\nDrug: Placebo for CK-3828136,Phase 1,...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...","['CK-3828136, CK-136']",Empty,0 days,Drug,78.0,None
44,44,"Study to Evaluate the Safety, Tolerability and...",Biofrontera Bioscience GmbH,Biofrontera Bioscience GmbH,Notyetrecruiting,2022-12-22,2022-12-22,Actinic Keratoses,Combination Product: BF-200 ALA and red light ...,Phase 3,...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",['Actinic keratosis\r\n \t \t ...,"['Keratosis, Actinic ...",0 days,Combination Product,165.0,"DB00132, DB03384, DB16183"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427263,427263,Phase III Randomized Study of Lucinactant in F...,Windtree Therapeutics,Windtree Therapeutics,Terminated,NaN,1999-10-19,Meconium Aspiration,Drug: Lucinactant\nOther: Standard Care,Phase 3,...,1999-10-19,2012-05-01,2012-05-03,Empty,['cardiovascular and respiratory diseases\r\n ...,['Meconium Aspiration Syndrome ...,NaN,Drug,69.0,DB04897
427418,427418,Phase II Study of Midodrine for Neurogenic Ort...,NaN,National Center for Research Resources (NCRR),Completed,1999-10-19,2005-06-24,Orthostatic Hypotension,Drug: midodrine,Phase 2,...,NaN,NaN,NaN,Empty,['cardiovascular and respiratory diseases\r\n ...,"['Hypotension, Orthostatic ...",2075 days,Drug,300.0,DB00211
427498,427498,Comparison of Buprenorphine and Methadone - 3,NaN,National Institute on Drug Abuse (NIDA),Completed,1999-09-21,2017-01-31,Opioid-Related Disorders,Drug: Buprenorphine,Phase 2,...,NaN,NaN,NaN,Empty,Empty,['Opioid-Related Disorders ...,6342 days,Drug,0.0,"DB00333, DB00921"
427499,427499,Evaluation of Benztropine for Cocaine Craving - 2,NaN,National Institute on Drug Abuse (NIDA),Completed,1999-09-21,2017-02-24,Cocaine-Related Disorders,Drug: Benztropine,Phase 2,...,NaN,NaN,NaN,"['Studies a U.S. FDA-regulated Drug Product:',...",Empty,['Cocaine-Related Disorders ...,6366 days,Drug,0.0,"DB00245, DB00907"


In [313]:
# Save the DataFrame as a CSV file
phasesCleandDF.to_csv('phasesCleandDFTotal.csv', index=False)
print("DataFrame saved as 'phasesCleandDFTotal.csv'")

DataFrame saved as 'phasesCleandDFTotal.csv'


In [312]:
# Count the number of None values in the column
num_none_values = phasesCleandDF["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(phasesCleandDF)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")

Percentage of None values in column 'DrugBank_ID': 30.10%


In [96]:
# Count the number of None values in the column
num_none_values = phasesCleandDF["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(phasesCleandDF)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")

Percentage of None values in column 'DrugBank_ID': 41.72%


In [554]:
# Create a copy of the original DataFrame
cleanPunctuation = phasesCleandDF.copy()

# Replace "-" in columns 'tet' and 'test'
cleanPunctuation['titles'] = cleanPunctuation['titles'].str.replace('-', '')
cleanPunctuation['treatment'] = cleanPunctuation['treatment'].str.replace('-', '')

In [560]:
%%time
# Create an empty list to store the names
names = []
cleanPunctuation["DrugBank_ID"] = ''

# Iterate over the rows of the Main DataFrame
for index, row in cleanPunctuation.iterrows():
    # Check if any nickname is present in the Last Name or First Name
    # Include a check for short names
    found_name = names_list_df.loc[names_list_df['Joined_Names'].apply(lambda x: any((f'{nickname.lower().replace('-', '')} ' if len(nickname) < 4 else nickname.lower().replace('-', '')) in row['titles'].lower() or (f' {nickname.lower().replace('-', '')} ' if len(nickname.lower()) < 4 else nickname.lower().replace('-', '')) in row['treatment'].lower() for nickname in x)), 'DrugBank_ID'].values.tolist()
    # If found, add the corresponding Name to the list
    if found_name:
        names.append(found_name[0])
        #phasesCleandDF.at[index, 'DrugBank_ID'] += found_name[0]
        cleanPunctuation.loc[index, 'DrugBank_ID'] += found_name[0]
    else:
        names.append(None)
        # phasesCleandDF.at[index, 'DrugBank_ID'] = None
        cleanPunctuation.loc[index, 'DrugBank_ID'] = None

# # Add the new column "Name" to the Main DataFrame
#phasesCleandDF['DrugBank_ID'] = names

CPU times: user 12h 41min 10s, sys: 41 s, total: 12h 41min 51s
Wall time: 12h 44min 18s


In [561]:
# Count the number of None values in the column
num_none_values = cleanPunctuation["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(cleanPunctuation)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")

Percentage of None values in column 'DrugBank_ID': 29.47%


In [565]:
# Save the DataFrame as a CSV file
cleanPunctuation.to_csv('cleanPunctuation.csv', index=False)
print("DataFrame saved as 'cleanPunctuation.csv'")

DataFrame saved as 'cleanPunctuation.csv'


### return from previous exercise are:
    1. cleanPunctuation @ cleanPunctuation.csv
    2. phasesCleandDF @ phasesCleandDFTotal.csv

### create a short subset for 2006-2022 and focusing on the top 20 companies

In [563]:
# Filter for the top 20 Companies
cleanTop20Companies = cleanPunctuation[cleanPunctuation['sponsor'].str.contains(r'Pete(r)?|Janssen|Johnson &amp; Johnson|Bristol-Myers Squibb|Astellas|Pfizer|Gilead|Takeda|AstraZeneca|Novartis|AbbVie|Eli Lilly|Genentech|Celgene|Glaxo|Sanofi|Roche|Eisai|Bayer|Boehringer Ingelheim|Novo Nordisk|Amgen|Merck', case=False, regex=True)]

cleanTop20Companies = cleanTop20Companies.drop(columns=['Unnamed: 0'])


# Reduce the dataset to 2006-2022
cleanTop20Companies2006 = cleanTop20Companies[(cleanTop20Companies['StudyStartDate'] >= '2006-01-01') & (cleanTop20Companies['StudyStartDate'] <= '2022-12-31')]


#Create a list/set of DB_IDs
ID_df = cleanTop20Companies2006[cleanTop20Companies2006['DrugBank_ID'].notnull()]

# Extract values from the 'ID' column into a list
values_list = list(chain.from_iterable(ID_df['DrugBank_ID'].str.split(', ')))

# Print the list of extracted values
#print(values_list)
id_list = list(set(values_list))
print(len(set(id_list)))

/var/folders/t4/yngm0l7s3lb4wk6mfrj9t87c0000gn/T/ipykernel_39898/831092445.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cleanTop20Companies = cleanPunctuation[cleanPunctuation['sponsor'].str.contains(r'Pete(r)?|Janssen|Johnson &amp; Johnson|Bristol-Myers Squibb|Astellas|Pfizer|Gilead|Takeda|AstraZeneca|Novartis|AbbVie|Eli Lilly|Genentech|Celgene|Glaxo|Sanofi|Roche|Eisai|Bayer|Boehringer Ingelheim|Novo Nordisk|Amgen|Merck', case=False, regex=True)]


1962


In [425]:
# Janssen, Johnson &amp; Johnson --> delete Wim Janssens  
# Bristol --> Bristol-Myers Squibb
# Astellas --> good 
# Pfizer --> good
# Gilead --> good
# Takeda --> good
# AstraZeneca --> good
# Novartis --> good
# AbbVie --> good | Abbot delete: Donald Mabbott AND Combined Military Hospital Abbottabad 
# Eli Lilly --> good
# Genentech --> good
# Celgene --> good
# Glaxo Smith Kline --> Glaxo to include Glaxo Wellcome
# Sanofi --> good
# Roche --> Hoffmann-La Roche
# Eisai --> good
# Bayer --> delete: Bayero University Kano, Nigeria
# Boehringer Ingelheim --> good
# Novo Nordisk --> good
# Amgen --> good
# Merck --> good
company = "Roche"

# Filter rows where "Tim" is present in the column "Name"
filtered_df = phasesCleandDF[phasesCleandDF['sponsor'].str.contains(company.lower(), case= False)]

# Count the occurrences of each unique value in the filtered column
name_counts = filtered_df['sponsor'].value_counts()

# Print the values and counts
print(f"Values and Counts in 'Name' column containing '{company.lower()}':")
print(name_counts)
print(len(name_counts))

Values and Counts in 'Name' column containing 'roche':
sponsor
Hoffmann-La Roche                              1216
University of Rochester                         153
University of Rochester NCORP Research Base       5
Rochester General Hospital                        4
Rochester Skin Lymphoma Medical Group, PLLC       1
Name: count, dtype: int64
5


In [314]:
# Filter for the top 20 Companies
top20Companies = phasesCleandDF[phasesCleandDF['sponsor'].str.contains(r'Pete(r)?|Janssen|Johnson &amp; Johnson|Bristol-Myers Squibb|Astellas|Pfizer|Gilead|Takeda|AstraZeneca|Novartis|AbbVie|Eli Lilly|Genentech|Celgene|Glaxo|Sanofi|Roche|Eisai|Bayer|Boehringer Ingelheim|Novo Nordisk|Amgen|Merck', case=False, regex=True)]

top20Companies = top20Companies.drop(columns=['Unnamed: 0'])

#Create a list/set of DB_IDs
ID_df = top20Companies[top20Companies['DrugBank_ID'].notnull()]

# Extract values from the 'ID' column into a list
values_list = list(chain.from_iterable(ID_df['DrugBank_ID'].str.split(', ')))

# Print the list of extracted values
#print(values_list)
id_list = list(set(values_list))
print(len(set(id_list)))

/var/folders/t4/yngm0l7s3lb4wk6mfrj9t87c0000gn/T/ipykernel_39898/700919212.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  top20Companies = phasesCleandDF[phasesCleandDF['sponsor'].str.contains(r'Pete(r)?|Janssen|Johnson &amp; Johnson|Bristol-Myers Squibb|Astellas|Pfizer|Gilead|Takeda|AstraZeneca|Novartis|AbbVie|Eli Lilly|Genentech|Celgene|Glaxo|Sanofi|Roche|Eisai|Bayer|Boehringer Ingelheim|Novo Nordisk|Amgen|Merck', case=False, regex=True)]


In [315]:
top20Companies

,titles,Responsible_Party,sponsor,RecruitmentStatus,FirstPosted,LastUpdatePosted,condition,treatment,phase,StudyType,...,FirstPosted2,LastUpdatePosted2,LastVerified,FDA,Keywords,AdditonalRelevantPoints,Post_updated_for,category,participants,DrugBank_ID
56,"A Study to Investigate the Safety, Tolerabilit...",AstraZeneca,AstraZeneca,Recruiting,2022-12-22,2022-12-22,Inflammatory Diseases,Drug: AZD673\nDrug: Placebo,Phase 1,Interventional (Clinical Trial),...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",['IRAK4 kinase inhibitor\r\n \t ...,Empty,0 days,Drug,133.0,None
70,Clinical Evaluation of a Multidose Preservativ...,"Johnson & Johnson Vision Care, Inc.","Johnson &amp; Johnson Vision Care, Inc.",Notyetrecruiting,2022-12-22,2022-12-22,Dry Eye,Drug: Investigational Lubricating Eye Drop in ...,Phase 2,Interventional (Clinical Trial),...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",Empty,['Dry Eye Syndromes ...,0 days,Drug,116.0,None
92,A Study to Evaluate the Safety and Pharmacokin...,Hoffmann-La Roche,Hoffmann-La Roche,Notyetrecruiting,2022-12-22,2022-12-22,PD-L1-selected Solid Tumors,Drug: Tiragolumab and Atezolizumab IV FDC,Phase 2,Interventional (Clinical Trial),...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",Empty,['Neoplasms Atezoli...,0 days,Drug,60.0,"DB11595, DB16409"
194,"A Study to Investigate the Safety, Tolerabilit...",GlaxoSmithKline,GlaxoSmithKline,Notyetrecruiting,2022-12-21,2022-12-21,"Hematologic Diseases\nAnaemia, Sickle Cell",Drug: GSK4172239D\nOther: Placebo,Phase 1,Interventional (Clinical Trial),...,2022-12-21,2022-12-21,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",['Hematologic diseases\r\n \t ...,"['Anemia, Sickle Cell ...",0 days,Drug,40.0,None
447,A Study to Learn How Well the Study Treatment ...,Bayer,Bayer,Notyetrecruiting,2022-12-19,2022-12-19,Atopic Dermatitis,Drug: Zabedosertib (BAY1834845)\nDrug: Placebo...,Phase 2,Interventional (Clinical Trial),...,2022-12-19,2022-12-19,2022-12-01,Empty,Empty,"['Dermatitis, Atopic ...",0 days,Drug,72.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423975,"A Comparison of BMS-232632 With Efavirenz, Eac...",NaN,Bristol-Myers Squibb,Completed,2001-08-31,2011-05-04,HIV Infections,Drug: Atazanavir\nDrug: Lamivudine/Zidovudine\...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,['Placebos\r\n \t \t ...,['HIV Infections Bl...,3533 days,Drug,NaN,DB01072
423990,Comparison of GW433908/Ritonavir to Nelfinavir...,NaN,Glaxo Wellcome,Unknown,NaN,2001-08-31,HIV Infections,Drug: Ritonavir\nDrug: Abacavir sulfate\nDrug:...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,['HIV-1\r\n \t \t Dru...,['HIV Infections Bl...,NaN,Drug,624.0,"DB00220, DB00503, DB00709, DB01048"
423995,Comparison of GW433908 and Nelfinavir in HIV P...,NaN,Glaxo Wellcome,Unknown,NaN,2001-08-31,HIV Infections,Drug: Abacavir sulfate\nDrug: Nelfinavir mesyl...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,['HIV-1\r\n \t \t Dru...,['HIV Infections Bl...,NaN,Drug,210.0,"DB00220, DB01048, DB01319"
424052,Effectiveness and Safety of Two Forms of Stavu...,NaN,Bristol-Myers Squibb,Completed,2001-08-31,2011-05-04,HIV Infections,Drug: Efavirenz\nDrug: Lamivudine\nDrug: Stavu...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,"['Drug Therapy, Combination\r\n \t ...",['HIV Infections Bl...,3533 days,Drug,730.0,DB00649


In [245]:
# Count the number of None values in the column
num_none_values = top20Companies["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(top20Companies)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")

Percentage of None values in column 'DrugBank_ID': 33.38%


In [566]:
#cleanTop20Companies

# Count the number of None values in the column
num_none_values = cleanTop20Companies["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(cleanTop20Companies)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")
print(num_none_values)
print(total_rows)

Percentage of None values in column 'DrugBank_ID': 19.39%
4695
24209


In [348]:
# Count the number of None values in the column
num_none_values = top20Companies["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(top20Companies)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")
print(num_none_values)
print(total_rows)

Percentage of None values in column 'DrugBank_ID': 20.36%
4928
24209


In [317]:
#Create a list/set of DB_IDs
ID_df = top20Companies[top20Companies['DrugBank_ID'].notnull()]

# Extract values from the 'ID' column into a list
values_list = list(chain.from_iterable(ID_df['DrugBank_ID'].str.split(', ')))

# Print the list of extracted values
#print(values_list)
id_list = list(set(values_list))
print(len(set(id_list)))


2618


In [567]:
# cleanTop20Companies
#Create a list/set of DB_IDs
ID_df = cleanTop20Companies[cleanTop20Companies['DrugBank_ID'].notnull()]

# Extract values from the 'ID' column into a list
values_list = list(chain.from_iterable(ID_df['DrugBank_ID'].str.split(', ')))

# Print the list of extracted values
#print(values_list)
id_list = list(set(values_list))
print(len(set(id_list)))


2115


In [255]:
# get common names
drugBankVocabulary = pd.read_csv('drugBankTotal.csv')

# Create a smaller DataFrame with only the IDs and Names from the list
trialed_drugs_df = drugBankVocabulary[drugBankVocabulary['DrugBank ID'].isin(id_list)][['DrugBank ID', 'Common name', 'Synonyms']]
#get approval from all 3 sources 
#try match them with common names? 

In [318]:
smaller_approval_df = pd.read_csv("smaller_approval_df.csv").drop(columns=['Unnamed: 0'])

In [322]:
Trade_name = smaller_approval_df["Generic name (Trade name)"].tolist()

In [329]:
import re
# Regular expression pattern to extract names prior to and within brackets
pattern = r'([^\(\)]+)\s*\(([^)]+)\)'

# Extract names from the list using regular expressions
extracted_names = []
for item in Trade_name:
    if isinstance(item, str):
        match = re.search(pattern, item)
        if match:
            name_parts = match.groups()
            extracted_names.extend([name_parts[0].strip(), name_parts[1].strip()])
        else:
            extracted_names.append(item)

In [331]:
len(extracted_names)

1321

In [395]:
extracted_names

['Daridorexant',
 'Quviviq',
 'Abrocitinib',
 'Cibinqo',
 'Tebentafusp',
 'Kimmtrak',
 'Faricimab',
 'Vabysmo',
 'Sutimlimab',
 'Enjaymo',
 'Mitapivat',
 'Pyrukynd',
 'Pacritinib',
 'Vonjo',
 'Ganaxolone',
 'Ztalmy',
 'Relatlimab; nivolumab',
 'Opdualag',
 'Lutetium Lu-177 vipivotide',
 '2022',
 'Oteseconazole',
 'Vivjoa',
 'Mavacamten',
 'Camzyos',
 'Vonoprazan; amoxicillin;',
 '2022',
 'Tirzepatide',
 'Mounjaro',
 'Tapinarof',
 'Vtama',
 'Vutrisiran',
 'Amvuttra',
 'Olipudase alfa',
 'Xenpozyme',
 'Spesolimab',
 'Spevigo',
 'DaxibotulinumtoxinA',
 'Daxxify',
 'Deucravacitinib',
 'Sotyktu',
 'Eflapegrastim',
 'Rolvedon',
 'Terlipressin',
 'Terlivaz',
 'Gadopiclenol',
 'Elucirem',
 'Omidenepag isopropyl',
 'Omlonti',
 'Sodium phenylbutyrate;',
 '2022',
 'Futibatinib',
 'Lytgobi',
 'Tremelimumab',
 'Imjudo',
 'Teclistamab',
 'Tecvayli',
 'Mirvetuximab soravtansine',
 '2022',
 'Teplizumab',
 'Tzield',
 'Olutasidenib',
 'Rezlidhia',
 'Adagrasib',
 'Krazati',
 'Lenacapavir',
 'Sunlenca',
 

In [396]:
%%time
# Create a new column "Nature_Approvals" in df
df['Nature_Approvals'] = None

# Iterate over the DataFrame and check for matches
for index, row in df.iterrows():
    for name in extracted_names:
        if isinstance(row['Common name'], str) and name.lower() in row['Common name'].lower():
            df.at[index, 'Nature_Approvals'] = name
        elif isinstance(row['Synonyms'], str) and name.lower() in row['Synonyms'].lower():
            df.at[index, 'Nature_Approvals'] = name
        elif isinstance(row['Brand_Names'], str) and name.lower() in row['Brand_Names'].lower():
            df.at[index, 'Nature_Approvals'] = name

CPU times: user 3min 14s, sys: 858 ms, total: 3min 15s
Wall time: 3min 17s


In [342]:
df

,DrugBank ID,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key,Brand_Names,DB_Approval,External_IDs,Nature_Approvals
0,DB00001,BIOD00024 | BTD00024,Lepirudin,138068-37-8,Y43GF64R34,"[Leu1, Thr2]-63-desulfohirudin | Desulfatohiru...",NaN,NaN,"Approved, Withdrawn",NaN,NA
1,DB00002,BIOD00071 | BTD00071,Cetuximab,205923-56-4,PQX0D8J21J,Cetuximab | Cétuximab | Cetuximabum,NaN,Erbitux,Approved,ABP-494 | BMS 564717 | C-225 | C225 | CMAB-009...,None
2,DB00003,BIOD00001 | BTD00001,Dornase alfa,143831-71-4,953A26OA1Y,Deoxyribonuclease (human clone 18-1 protein mo...,NaN,Pulmozyme,Approved,NaN,NA
3,DB00004,BIOD00084 | BTD00084,Denileukin diftitox,173146-27-5,25E79B5CTM,Denileukin | Denileukin diftitox | Interleukin...,NaN,NaN,"Approved, Investigational",DAB-389IL2 | DAB389 IL2 | DAB389IL2 | LY-33534...,None
4,DB00005,BIOD00052 | BTD00052,Etanercept,185243-69-0,OP401G7OJC,Etanercept | etanercept-szzs | etanercept-ykro...,NaN,Enbrel | Eticovo,"Approved, Investigational",CHS-0214 | DWP-422 | ENIA-11 | GP-2015 | GP201...,NA
...,...,...,...,...,...,...,...,...,...,...,...
16570,DB18707,NaN,AVL-3288,917837-54-8,VA80VAX4WF,(z)-3-(4-chloroanilino)-n-(4-chlorophenyl)-2-(...,VMAKIACTLSBBIY-BOPFTXTBSA-N,NaN,Investigational,AVL-3288 | CCMI | UCI-4083 | XY-4083,NA
16571,DB18708,NaN,Alogabat,2230009-48-8,0HPA4GK3UH,"3-pyridazinecarboxamide, 6-((5-methyl-3-(6-met...",ACZCJTHHWMBFKC-UHFFFAOYSA-N,NaN,Investigational,RG7816,None
16572,DB18709,NaN,Ropsacitinib,2127109-84-4,HY5SOV7O0Q,"Cyclobutaneacetonitrile, 3-cyano-1-(4-(6-(1-me...",XPLZTJWZDBFWDE-OYOVHJISSA-N,NaN,Investigational,PF-06826647,None
16573,DB18710,NaN,ABBV-706,NaN,NaN,ABBV 706 | ABBV-706 | ABBV706,NaN,NaN,Investigational,NaN,None


In [453]:
drugBankFeb29 = pd.read_csv("drugBankFeb29.csv")

In [ ]:
drugBankFeb29

In [ ]:
top20Companies

In [467]:
%%time

# Reduce the dataset to 2006-2022
companyDataFrom2006 = top20Companies[(top20Companies['StudyStartDate'] >= '2006-01-01') & (top20Companies['StudyStartDate'] <= '2022-12-31')]
            

CPU times: user 118 ms, sys: 678 ms, total: 796 ms
Wall time: 1.69 s


In [562]:
#Create a list/set of DB_IDs
ID_df = companyDataFrom2006[companyDataFrom2006['DrugBank_ID'].notnull()]

# Extract values from the 'ID' column into a list
values_list = list(chain.from_iterable(ID_df['DrugBank_ID'].str.split(', ')))

# Print the list of extracted values
#print(values_list)
id_list = list(set(values_list))
print(len(set(id_list)))


2357


In [ ]:
id_list

In [468]:
# Group by 'DrugBank_ID' and 'sponsor', count occurrences, then pivot the table
grouped_df = companyDataFrom2006.groupby(['DrugBank_ID', 'sponsor']).size().reset_index(name='count')
pivot_df_sponsor = grouped_df.pivot(index='DrugBank_ID', columns='sponsor', values='count').fillna(0)


In [461]:
### get a a list of db IDs
testDrugBankIDs = companyDataFrom2006["DrugBank_ID"].tolist()

# Remove all None values from the list
testDrugBankIDs = [value for value in testDrugBankIDs if value is not None]

# Split each string and save them as individual strings in a new list
testDrugBankIDs = [value.strip() for sublist in testDrugBankIDs for value in sublist.split(',')]

## reduce the number of approved that are not in the dataset
## create WIP? 

In [500]:
# Split the 'name' column by comma and stack the DataFrame
df_split = companyDataFrom2006.assign(name=companyDataFrom2006['DrugBank_ID'].str.split(', ')).explode('DrugBank_ID')

# Filter DataFrame based on IDs from the list
filtered_df = df_split[df_split['DrugBank_ID'].isin(id_list)]

# Group by 'ID' and 'name' in the filtered DataFrame, count occurrences, then pivot the table
# grouped_df = filtered_df.groupby(['sponsor', 'DrugBank_ID']).size().reset_index(name='count')
# pivot_company_df = grouped_df.pivot(index='sponsor', columns='DrugBank_ID', values='count').fillna(0)

grouped_df = filtered_df.groupby(['sponsor', 'DrugBank_ID']).size().reset_index(name='count')
pivot_df = grouped_df.pivot(index='sponsor', columns='DrugBank_ID', values='count').fillna(0)

grouped_df = filtered_df.groupby(['phase', 'DrugBank_ID']).size().reset_index(name='count')
pivot_phase_df = grouped_df.pivot(index='phase', columns='DrugBank_ID', values='count').fillna(0)

#pivot_company_df = pivot_company_df.T
pivot_phase_df = pivot_phase_df.T
pivot_df = pivot_df.T

In [501]:
pivot_df

sponsor,AbbVie,Amgen,Astellas,AstraZeneca,Bayer,Boehringer Ingelheim,Bristol-Myers Squibb,Celgene,Eisai,Eli Lilly,...,Gilead,Glaxo Smith Kline,Hoffmann-La Roche,Johnson & Johnson,Merck,Novartis,Novo Nordisk,Pfizer,Sanofi,Takeda
DrugBank_ID,,,,,,,,,,,,,,,,,,,,,
DB00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00005,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0
DB00007,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
DB00008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB00009,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB18474,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB18549,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DB18662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
companyDataFrom2006

In [477]:
drugBankFeb29.columns

Index(['DrugBank ID', 'Accession Numbers', 'Common name', 'CAS', 'UNII',
       'Synonyms', 'Standard InChI Key', 'Brand_Names', 'DB_Approval',
       'External_IDs', 'brand_names_nature_approval',
       'generic_name_nature_approval', 'generic_approval_year',
       'brand_approval_year'],
      dtype='object')

In [496]:
pivot_company_df["DrugBank_Approval"] = None
pivot_company_df["Generic_approval_name"] = None
pivot_company_df["Generic_approval_year"] = None

for id in pivot_company_df.index:
    for index, row in drugBankFeb29.iterrows():
        if id == row["DrugBank ID"]:
            pivot_company_df.loc[id, 'DrugBank_Approval'] = row["DB_Approval"]
            pivot_company_df.loc[id, 'Generic_approval_name'] = row["generic_name_nature_approval"]
            pivot_company_df.loc[id, 'Generic_approval_year'] = row["generic_approval_year"]

In [502]:
# Delete rows with only None or NaN values
pivot_company_df = pivot_company_df.dropna(how='all') #somehow the 

In [504]:
pivot_company_df = pivot_company_df.replace({np.nan: None})

In [ ]:
pivot_company_df.loc["DB16033"] #curious 
clean_pivot_company_df = pivot_company_df.drop('DB16033')

In [ ]:
# Create smaller DataFrames for each name with non-zero values and specific columns
individual_companies = {}
for column in clean_pivot_company_df.columns:
    if column in ['DrugBank_Approval', 'Generic_approval_name','Generic_approval_year'] or (clean_pivot_company_df[column] != 0).any():
        non_zero_indexes = clean_pivot_company_df.index[clean_pivot_company_df[column] != 0]
        smaller_df = clean_pivot_company_df.loc[non_zero_indexes, ['DrugBank_Approval', 'Generic_approval_name','Generic_approval_year', column]]
        individual_companies[column] = smaller_df

# Print the smaller DataFrames for each name
# for name, smaller_df in individual_companies.items():
#     print(f"DataFrame for {name}:")
#     print(smaller_df)

In [549]:
def PoSCalculator(TARGETDF):
    # Compute the total 
    approved_drugs = []
    approved_nature_durgs = []
    just_approved = []

    for index, row in TARGETDF.iterrows():
        if "Approved" in row["DrugBank_Approval"] and row["Generic_approval_name"] == None:
            approved_drugs.append(index)
        elif row["Generic_approval_name"] != None:
            approved_nature_durgs.append(index)
        if "Approved" == row["DrugBank_Approval"]:
            just_approved.append(index)
    return approved_drugs,approved_nature_durgs,just_approved

In [552]:
# Delete the last 3 elements from the dictionary
for _ in range(3):
    individual_companies.popitem()

In [553]:
for company,data in individual_companies.items():
    print(company)
    approved_drugs,approved_nature_durgs,just_approved = PoSCalculator(data)
    print(f"{company} conducted clinical trials on {len(data)} compounds found in DrugBank. Of those {len(approved_drugs)} were approved outside of the observation period")
    print(f"In total {len(approved_nature_durgs)} compounds were approved by the FDA (acc. to nature publications)")
    print(f"The PoS is {len(approved_nature_durgs)/(len(data)-len(approved_drugs))*100} or {len(approved_nature_durgs)/(len(data)-len(just_approved))*100} if only focused on the soley 'Approved' compounds")
    print()
    #average sucess rate 1.1

AbbVie
AbbVie conducted clinical trials on 57 compounds found in DrugBank. Of those 19 were approved outside of the observation period
In total 11 compounds were approved by the FDA (acc. to nature publications)
The PoS is 28.947368421052634 or 23.404255319148938 if only focused on the soley 'Approved' compounds

Amgen
Amgen conducted clinical trials on 56 compounds found in DrugBank. Of those 15 were approved outside of the observation period
In total 17 compounds were approved by the FDA (acc. to nature publications)
The PoS is 41.46341463414634 or 37.77777777777778 if only focused on the soley 'Approved' compounds

Astellas
Astellas conducted clinical trials on 67 compounds found in DrugBank. Of those 34 were approved outside of the observation period
In total 11 compounds were approved by the FDA (acc. to nature publications)
The PoS is 33.33333333333333 or 22.0 if only focused on the soley 'Approved' compounds

AstraZeneca
AstraZeneca conducted clinical trials on 145 compounds fou

In [505]:
pivot_company_df

sponsor,AbbVie,Amgen,Astellas,AstraZeneca,Bayer,Boehringer Ingelheim,Bristol-Myers Squibb,Celgene,Eisai,Eli Lilly,...,Johnson & Johnson,Merck,Novartis,Novo Nordisk,Pfizer,Sanofi,Takeda,DrugBank_Approval,Generic_approval_name,Generic_approval_year
DrugBank_ID,,,,,,,,,,,,,,,,,,,,,
DB00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Approved, Investigational",None,None
DB00005,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,"Approved, Investigational",None,None
DB00007,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Approved, Investigational",None,None
DB00008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Approved, Investigational",None,None
DB00009,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Approved, Investigational",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB18474,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Investigational,None,None
DB18549,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Investigational,None,None
DB18662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Investigational,None,None


In [ ]:
pivot_company_df.loc["DB16033"] #curious 
clean_pivot_company_df = pivot_company_df.drop('DB16033')

In [534]:
# Get all unique values from the 'Category' column
unique_values = clean_pivot_company_df['DrugBank_Approval'].unique()

# Print the unique values
print("Unique values in the 'Category' column:")
print(unique_values)

Unique values in the 'Category' column:
['Approved, Investigational' 'Approved' 'Approved, Experimental'
 'Approved, Investigational, Vet approved'
 'Approved, Investigational, Withdrawn' 'Approved, Vet approved'
 'Approved, Investigational, Nutraceutical' 'Approved, Nutraceutical'
 'Approved, Nutraceutical, Vet approved' 'Investigational, Nutraceutical'
 'Approved, Illicit' 'Approved, Vet approved, Withdrawn'
 'Approved, Illicit, Investigational'
 'Approved, Illicit, Investigational, Vet approved' 'Experimental'
 'Approved, Withdrawn' 'Approved, Illicit, Investigational, Withdrawn'
 'Experimental, Investigational' 'Approved, Experimental, Investigational'
 'Investigational' 'Investigational, Withdrawn'
 'Investigational, Vet approved'
 'Experimental, Investigational, Withdrawn']


In [540]:
# Compute the total 
approved_drugs = []
approved_nature_durgs = []
investigated_drugs = []
just_drugs = []

for index, row in clean_pivot_company_df.iterrows():
    if "Approved" in row["DrugBank_Approval"] and row["Generic_approval_name"] == None:
        approved_drugs.append(index)
    elif row["Generic_approval_name"] != None:
        approved_nature_durgs.append(index)
    if "Approved" == row["DrugBank_Approval"]:
        just_drugs.append(index)

print(len(pivot_company_df))
print(len(approved_drugs))
print(len(approved_nature_durgs))
print(len(just_drugs))

1580
457
297
266


In [541]:
297/(1580-266)*100

22.602739726027394

In [525]:
297/(1580-457)*100

26.447016918967055

In [520]:
for index, row in pivot_company_df.iterrows():
    try:
        if "Approved" in row["DrugBank_Approval"] and row["Generic_approval_name"] == None:
            #print(row["Generic_approval_name"])
            pass
    except:
        print(index)

DB16033


In [458]:
# Count the number of None values in the column
num_none_values = companyDataFrom2006["DrugBank_ID"].isna().sum()

# Calculate the percentage of None values
total_rows = len(companyDataFrom2006)
percentage_none_values = (num_none_values / total_rows) * 100

# Print the percentage of None values
print(f"Percentage of None values in column 'DrugBank_ID': {percentage_none_values:.2f}%")
print(num_none_values)
print(total_rows)

Percentage of None values in column 'DrugBank_ID': 20.96%
4197
20020


In [ ]:
# Group by 'DrugBank_ID' and 'sponsor', count occurrences, then pivot the table
grouped_df = top20Companies.groupby(['DrugBank_ID', 'sponsor']).size().reset_index(name='count')
pivot_df_sponsor = grouped_df.pivot(index='DrugBank_ID', columns='sponsor', values='count').fillna(0)

print(pivot_df_sponsor)

In [359]:
grouped_df = top20Companies.groupby(['DrugBank_ID', 'phase']).size().reset_index(name='count')
pivot_df_phase = grouped_df.pivot(index='DrugBank_ID', columns='phase', values='count').fillna(0)

print(pivot_df_phase)

phase                                               Phase 1  Phase 2  Phase 3
DrugBank_ID                                                                  
DB00002                                                 0.0      1.0      0.0
DB00002, DB00072, DB00456, DB09274, DB16092             1.0      0.0      0.0
DB00002, DB00112, DB00441, DB00456                      0.0      1.0      0.0
DB00002, DB00112, DB00456, DB00526, DB00762, DB...      0.0      1.0      0.0
DB00002, DB00112, DB00456, DB00762, DB09274             1.0      0.0      0.0
...                                                     ...      ...      ...
DB18474                                                 1.0      0.0      0.0
DB18549                                                 1.0      0.0      0.0
DB18662                                                 2.0      0.0      0.0
DB18693                                                 1.0      0.0      0.0
DB18709                                                 2.0     

In [382]:
pivot_df_phase

phase,Phase 1,Phase 2,Phase 3,DB_Approval
DrugBank_ID,,,,
DB00002,0.0,1.0,0.0,
"DB00002, DB00072, DB00456, DB09274, DB16092",1.0,0.0,0.0,
"DB00002, DB00112, DB00441, DB00456",0.0,1.0,0.0,
"DB00002, DB00112, DB00456, DB00526, DB00762, DB12250",0.0,1.0,0.0,
"DB00002, DB00112, DB00456, DB00762, DB09274",1.0,0.0,0.0,
...,...,...,...,...
DB18474,1.0,0.0,0.0,
DB18549,1.0,0.0,0.0,
DB18662,2.0,0.0,0.0,


In [376]:
df

,DrugBank ID,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key,Brand_Names,DB_Approval,External_IDs
0,DB00001,BIOD00024 | BTD00024,Lepirudin,138068-37-8,Y43GF64R34,"[Leu1, Thr2]-63-desulfohirudin | Desulfatohiru...",NaN,NaN,"Approved, Withdrawn",NaN
1,DB00002,BIOD00071 | BTD00071,Cetuximab,205923-56-4,PQX0D8J21J,Cetuximab | Cétuximab | Cetuximabum,NaN,Erbitux,Approved,ABP-494 | BMS 564717 | C-225 | C225 | CMAB-009...
2,DB00003,BIOD00001 | BTD00001,Dornase alfa,143831-71-4,953A26OA1Y,Deoxyribonuclease (human clone 18-1 protein mo...,NaN,Pulmozyme,Approved,NaN
3,DB00004,BIOD00084 | BTD00084,Denileukin diftitox,173146-27-5,25E79B5CTM,Denileukin | Denileukin diftitox | Interleukin...,NaN,NaN,"Approved, Investigational",DAB-389IL2 | DAB389 IL2 | DAB389IL2 | LY-33534...
4,DB00005,BIOD00052 | BTD00052,Etanercept,185243-69-0,OP401G7OJC,Etanercept | etanercept-szzs | etanercept-ykro...,NaN,Enbrel | Eticovo,"Approved, Investigational",CHS-0214 | DWP-422 | ENIA-11 | GP-2015 | GP201...
...,...,...,...,...,...,...,...,...,...,...
16570,DB18707,NaN,AVL-3288,917837-54-8,VA80VAX4WF,(z)-3-(4-chloroanilino)-n-(4-chlorophenyl)-2-(...,VMAKIACTLSBBIY-BOPFTXTBSA-N,NaN,Investigational,AVL-3288 | CCMI | UCI-4083 | XY-4083
16571,DB18708,NaN,Alogabat,2230009-48-8,0HPA4GK3UH,"3-pyridazinecarboxamide, 6-((5-methyl-3-(6-met...",ACZCJTHHWMBFKC-UHFFFAOYSA-N,NaN,Investigational,RG7816
16572,DB18709,NaN,Ropsacitinib,2127109-84-4,HY5SOV7O0Q,"Cyclobutaneacetonitrile, 3-cyano-1-(4-(6-(1-me...",XPLZTJWZDBFWDE-OYOVHJISSA-N,NaN,Investigational,PF-06826647
16573,DB18710,NaN,ABBV-706,NaN,NaN,ABBV 706 | ABBV-706 | ABBV706,NaN,NaN,Investigational,NaN


In [366]:
# Split the 'name' column by comma and stack the DataFrame
df_split = top20Companies.assign(name=top20Companies['DrugBank_ID'].str.split(', ')).explode('DrugBank_ID')

# Filter DataFrame based on IDs from the list
filtered_df = df_split[df_split['DrugBank_ID'].isin(id_list)]

# Group by 'ID' and 'name' in the filtered DataFrame, count occurrences, then pivot the table
grouped_df = filtered_df.groupby(['phase', 'DrugBank_ID']).size().reset_index(name='count')
pivot_df = grouped_df.pivot(index='phase', columns='DrugBank_ID', values='count').fillna(0)

pivot_df = pivot_df.T

In [373]:
pivot_df = pivot_df.T

In [374]:
pivot_df

phase,Phase 1,Phase 2,Phase 3
DrugBank_ID,,,
DB00002,0.0,1.0,0.0
DB00004,1.0,2.0,0.0
DB00005,3.0,2.0,14.0
DB00007,0.0,3.0,1.0
DB00008,0.0,1.0,2.0
...,...,...,...
DB18474,1.0,0.0,0.0
DB18549,1.0,0.0,0.0
DB18662,2.0,0.0,0.0


In [386]:
for index,row in pivot_df[:5].iterrows():
    print(index)

DB00002
DB00004
DB00005
DB00007
DB00008


In [391]:
for index2, row2 in df.iterrows():
    if "DB00002" == row2["DrugBank ID"]:
        print(row2["DB_Approval"])

Approved


In [398]:
df

,DrugBank ID,Accession Numbers,Common name,CAS,UNII,Synonyms,Standard InChI Key,Brand_Names,DB_Approval,External_IDs,Nature_Approvals
0,DB00001,BIOD00024 | BTD00024,Lepirudin,138068-37-8,Y43GF64R34,"[Leu1, Thr2]-63-desulfohirudin | Desulfatohiru...",NaN,NaN,"Approved, Withdrawn",NaN,NA
1,DB00002,BIOD00071 | BTD00071,Cetuximab,205923-56-4,PQX0D8J21J,Cetuximab | Cétuximab | Cetuximabum,NaN,Erbitux,Approved,ABP-494 | BMS 564717 | C-225 | C225 | CMAB-009...,None
2,DB00003,BIOD00001 | BTD00001,Dornase alfa,143831-71-4,953A26OA1Y,Deoxyribonuclease (human clone 18-1 protein mo...,NaN,Pulmozyme,Approved,NaN,NA
3,DB00004,BIOD00084 | BTD00084,Denileukin diftitox,173146-27-5,25E79B5CTM,Denileukin | Denileukin diftitox | Interleukin...,NaN,NaN,"Approved, Investigational",DAB-389IL2 | DAB389 IL2 | DAB389IL2 | LY-33534...,None
4,DB00005,BIOD00052 | BTD00052,Etanercept,185243-69-0,OP401G7OJC,Etanercept | etanercept-szzs | etanercept-ykro...,NaN,Enbrel | Eticovo,"Approved, Investigational",CHS-0214 | DWP-422 | ENIA-11 | GP-2015 | GP201...,NA
...,...,...,...,...,...,...,...,...,...,...,...
16570,DB18707,NaN,AVL-3288,917837-54-8,VA80VAX4WF,(z)-3-(4-chloroanilino)-n-(4-chlorophenyl)-2-(...,VMAKIACTLSBBIY-BOPFTXTBSA-N,NaN,Investigational,AVL-3288 | CCMI | UCI-4083 | XY-4083,NA
16571,DB18708,NaN,Alogabat,2230009-48-8,0HPA4GK3UH,"3-pyridazinecarboxamide, 6-((5-methyl-3-(6-met...",ACZCJTHHWMBFKC-UHFFFAOYSA-N,NaN,Investigational,RG7816,None
16572,DB18709,NaN,Ropsacitinib,2127109-84-4,HY5SOV7O0Q,"Cyclobutaneacetonitrile, 3-cyano-1-(4-(6-(1-me...",XPLZTJWZDBFWDE-OYOVHJISSA-N,NaN,Investigational,PF-06826647,None
16573,DB18710,NaN,ABBV-706,NaN,NaN,ABBV 706 | ABBV-706 | ABBV706,NaN,NaN,Investigational,NaN,None


In [399]:
%%time
pivot_df['DrugBank_Approval'] = None
pivot_df['Nature_Approvals'] = None

for index,row in pivot_df.iterrows():
    for index2, row2 in df.iterrows():
        if index == row2["DrugBank ID"]:
            pivot_df.loc[index, 'DrugBank_Approval'] = row2["DB_Approval"]
            pivot_df.loc[index, 'Nature_Approvals'] = row2["Nature_Approvals"]

CPU times: user 23min 12s, sys: 8.9 s, total: 23min 21s
Wall time: 24min 7s


In [407]:
pivot_df

phase,Phase 1,Phase 2,Phase 3,DB_Approval,DrugBank_Approval,Nature_Approvals
DrugBank_ID,,,,,,
DB00002,0.0,1.0,0.0,Approved,Approved,None
DB00004,1.0,2.0,0.0,"Approved, Investigational","Approved, Investigational",None
DB00005,3.0,2.0,14.0,"Approved, Investigational","Approved, Investigational",None
DB00007,0.0,3.0,1.0,"Approved, Investigational","Approved, Investigational",None
DB00008,0.0,1.0,2.0,"Approved, Investigational","Approved, Investigational",None
...,...,...,...,...,...,...
DB18474,1.0,0.0,0.0,Investigational,Investigational,None
DB18549,1.0,0.0,0.0,Investigational,Investigational,None
DB18662,2.0,0.0,0.0,Investigational,Investigational,None


In [409]:
# Filter and print non-None or non-NA values from the specified column
# Replace "NA" values with None in the specified column
pivot_df["Nature_Approvals"] = pivot_df["Nature_Approvals"].replace('NA', None)
non_missing_values = pivot_df[pivot_df["Nature_Approvals"].notna()]["Nature_Approvals"].tolist()
print(non_missing_values)
print(len(set(non_missing_values)))

['Collagenase clostridium', 'Hizentra', 'Perjeta', 'IncobotulinumtoxinA', 'Tbo-Filgrastim', '2011', 'Ranexa', '2011', 'Eraxis', 'Benzyl alcohol', '2008', 'Rifamycin', 'Ella', 'Ella', 'Ella', 'Gattex', 'Pylera', '2011', 'Tekturna', 'Kisqali', 'Rifamycin', 'Amitiza', '2011', 'Sprycel', 'Altabax', 'Soliris', 'Tykerb', 'Januvia', 'Dacogen', 'Noxafil', 'Prezista', 'Telbivudine', 'Invega', 'Sutent', 'Vectibix', 'Lucentis', 'Chantix', 'Ella', 'Omnaris', 'Afinitor', 'Uridine', 'Daliresp', 'Zolinza', 'Selzentry', 'Ixempra', 'Uloric', 'Multaq', 'Tasigna', 'Lorcaserin', 'Fanapt', 'Esbriet', 'Ezogabine', 'Eucrisa', 'Cotellic', 'Caprelsa', 'Nplate', 'Telaprevir', 'Briviact', 'Cyramza', 'Otezla', 'Stelara', 'Zytiga', 'Inotuzumab ozogamicin', 'Cariprazine', 'Lartruvo', 'Cablivi', 'Ilaris', 'Istodax', 'Yervoy', 'Firazyr', 'Rufinamide', 'Nesina', 'Tapentadol', 'Bridion', 'Effient', 'Eltrombopag', 'Doripenem', 'Regadenoson', 'Dalvance', 'Xarelto', 'Livtencity', 'Actemra', 'Torisel', 'Simeprevir', 'Farxi

In [413]:
# Count rows where 'DrugBank_Approval' includes 'Approved' and 'Nature_Approvals' is None
count_rows = len(pivot_df[(pivot_df['DrugBank_Approval'].str.contains('Approved')) & (pivot_df['Nature_Approvals'].isnull())])

print("Number of rows where 'DB' includes 'Approved' and 'Nature_Approvals' is None:", count_rows)

Number of rows where 'DB' includes 'Approved' and 'Nature_Approvals' is None: 539


In [415]:
1773-539

1234

In [416]:
293/1234*100

23.743922204213938

In [434]:
# Filter for the top 20 Companies
top20Companies = phasesCleandDF[phasesCleandDF['sponsor'].str.contains(r'Janssen|Johnson &amp; Johnson|Bristol-Myers Squibb|Astellas|Pfizer|Gilead|Takeda|AstraZeneca|Novartis|AbbVie|Eli Lilly|Genentech|Celgene|Glaxo|Sanofi|Roche|Eisai|Bayer|Boehringer Ingelheim|Novo Nordisk|Amgen|Merck', case=False, regex=True)]

top20Companies = top20Companies.drop(columns=['Unnamed: 0'])

In [436]:
# Get and print unique values from the specified column
unique_values = top20Companies["sponsor"].unique()
print(unique_values)
print(len(top20Companies))

['AstraZeneca' 'Johnson &amp; Johnson Vision Care, Inc.'
 'Hoffmann-La Roche' 'GlaxoSmithKline' 'Bayer' 'Pfizer'
 'Novartis Pharmaceuticals' 'Janssen Research &amp; Development, LLC'
 'AbbVie' 'Amgen' 'Sanofi Pasteur, a Sanofi Company'
 'Bristol-Myers Squibb' 'Eli Lilly and Company' 'Sanofi' 'Gilead Sciences'
 'Genentech, Inc.' 'Novo Nordisk A/S'
 'Astellas Pharma Global Development, Inc.' 'Boehringer Ingelheim'
 'Merck Sharp &amp; Dohme LLC' 'Eisai Limited' 'Takeda' 'Eisai Co., Ltd.'
 'Kadmon, a Sanofi Company' 'Janssen Pharmaceutical K.K.'
 'Janssen Pharmaceutica N.V., Belgium'
 'Merck Healthcare KGaA, Darmstadt, Germany, an affiliate of Merck KGaA, Darmstadt, Germany'
 'Kite, A Gilead Company' 'Eisai Inc.' 'Astellas Pharma Inc'
 'Amunix, a Sanofi Company' 'Janssen Scientific Affairs, LLC'
 'Janssen-Cilag Ltd.' 'Janssen Vaccines &amp; Prevention B.V.'
 'University of Rochester' 'Genzyme, a Sanofi Company' 'Celgene'
 'Johnson &amp; Johnson Consumer Inc. (J&amp;JCI)'
 'Astellas Pharma 

In [441]:
# delete: Bayero University Kano, Nigeria, delete Wim Janssens  
print(len(top20Companies))
top20Companies = top20Companies[~top20Companies["sponsor"].str.contains("Wim Janssens|Bayero University Kano, Nigeria|University of Rochester|University of Rochester NCORP Research Base|Rochester General Hospital|Rochester Skin Lymphoma Medical Group, PLLC")]
print(len(top20Companies))

24045
23882


In [442]:
# Janssen, Johnson &amp; Johnson --> delete Wim Janssens  
# Bristol --> Bristol-Myers Squibb
# Astellas --> good 
# Pfizer --> good
# Gilead --> good
# Takeda --> good
# AstraZeneca --> good
# Novartis --> good
# AbbVie --> good | Abbot delete: Donald Mabbott AND Combined Military Hospital Abbottabad 
# Eli Lilly --> good
# Genentech --> good
# Celgene --> good
# Glaxo Smith Kline --> Glaxo to include Glaxo Wellcome
# Sanofi --> good
# Roche --> Hoffmann-La Roche
# Eisai --> good
# Bayer --> delete: Bayero University Kano, Nigeria
# Boehringer Ingelheim --> good
# Novo Nordisk --> good
# Amgen --> good
# Merck --> good
# Unifiy sponsor names
top20Companies["sponsor"] = top20Companies["sponsor"].apply(lambda x: "Johnson & Johnson" if "janssen" in x.lower() else(
    "Johnson & Johnson" if "Johnson &amp; Johnson" in x else(
        "Bristol-Myers Squibb" if "Bristol-Myers Squibb" in x else(
            "Pfizer" if "Pfizer" in x else(
                "Gilead" if "Gilead" in x else(
                    "Astellas" if "Astellas" in x else(
                        "Takeda" if "Takeda" in x else(
                            "AstraZeneca" if "AstraZeneca" in x else(
                                "Novartis" if "Novartis" in x else(
                                    "AbbVie" if "AbbVie" in x else(
                                        "Eli Lilly" if "Eli Lilly" in x else(
                                            "Genentech" if "Genentech" in x else(
                                                "Celgene" if "Celgene" in x else(
                                                    "Glaxo Smith Kline" if "Glaxo" in x else(
                                                        "Sanofi" if "Sanofi" in x else(
                                                            "Hoffmann-La Roche" if "Hoffmann-La Roche" in x else(
                                                                "Eisai" if "Eisai" in x else(
                                                                    "Bayer" if "Bayer" in x else(
                                                                        "Boehringer Ingelheim" if "Boehringer Ingelheim" in x else(
                                                                            "Novo Nordisk" if "Novo Nordisk" in x else(
                                                                                "Amgen" if "Amgen" in x else(
                                                                                    "Merck" if "Merck" in x else x
                                                                                    ))))))))))))))))))))))

/var/folders/t4/yngm0l7s3lb4wk6mfrj9t87c0000gn/T/ipykernel_39898/2602758649.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top20Companies["sponsor"] = top20Companies["sponsor"].apply(lambda x: "Johnson & Johnson" if "janssen" in x.lower() else(


In [443]:
top20Companies

,titles,Responsible_Party,sponsor,RecruitmentStatus,FirstPosted,LastUpdatePosted,condition,treatment,phase,StudyType,...,FirstPosted2,LastUpdatePosted2,LastVerified,FDA,Keywords,AdditonalRelevantPoints,Post_updated_for,category,participants,DrugBank_ID
56,"A Study to Investigate the Safety, Tolerabilit...",AstraZeneca,AstraZeneca,Recruiting,2022-12-22,2022-12-22,Inflammatory Diseases,Drug: AZD673\nDrug: Placebo,Phase 1,Interventional (Clinical Trial),...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",['IRAK4 kinase inhibitor\r\n \t ...,Empty,0 days,Drug,133.0,None
70,Clinical Evaluation of a Multidose Preservativ...,"Johnson & Johnson Vision Care, Inc.",Johnson & Johnson,Notyetrecruiting,2022-12-22,2022-12-22,Dry Eye,Drug: Investigational Lubricating Eye Drop in ...,Phase 2,Interventional (Clinical Trial),...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",Empty,['Dry Eye Syndromes ...,0 days,Drug,116.0,None
92,A Study to Evaluate the Safety and Pharmacokin...,Hoffmann-La Roche,Hoffmann-La Roche,Notyetrecruiting,2022-12-22,2022-12-22,PD-L1-selected Solid Tumors,Drug: Tiragolumab and Atezolizumab IV FDC,Phase 2,Interventional (Clinical Trial),...,2022-12-22,2022-12-22,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",Empty,['Neoplasms Atezoli...,0 days,Drug,60.0,"DB11595, DB16409"
194,"A Study to Investigate the Safety, Tolerabilit...",GlaxoSmithKline,Glaxo Smith Kline,Notyetrecruiting,2022-12-21,2022-12-21,"Hematologic Diseases\nAnaemia, Sickle Cell",Drug: GSK4172239D\nOther: Placebo,Phase 1,Interventional (Clinical Trial),...,2022-12-21,2022-12-21,2022-12-01,"['Studies a U.S. FDA-regulated Drug Product:',...",['Hematologic diseases\r\n \t ...,"['Anemia, Sickle Cell ...",0 days,Drug,40.0,None
447,A Study to Learn How Well the Study Treatment ...,Bayer,Bayer,Notyetrecruiting,2022-12-19,2022-12-19,Atopic Dermatitis,Drug: Zabedosertib (BAY1834845)\nDrug: Placebo...,Phase 2,Interventional (Clinical Trial),...,2022-12-19,2022-12-19,2022-12-01,Empty,Empty,"['Dermatitis, Atopic ...",0 days,Drug,72.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423975,"A Comparison of BMS-232632 With Efavirenz, Eac...",NaN,Bristol-Myers Squibb,Completed,2001-08-31,2011-05-04,HIV Infections,Drug: Atazanavir\nDrug: Lamivudine/Zidovudine\...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,['Placebos\r\n \t \t ...,['HIV Infections Bl...,3533 days,Drug,NaN,DB01072
423990,Comparison of GW433908/Ritonavir to Nelfinavir...,NaN,Glaxo Smith Kline,Unknown,NaN,2001-08-31,HIV Infections,Drug: Ritonavir\nDrug: Abacavir sulfate\nDrug:...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,['HIV-1\r\n \t \t Dru...,['HIV Infections Bl...,NaN,Drug,624.0,"DB00220, DB00503, DB00709, DB01048"
423995,Comparison of GW433908 and Nelfinavir in HIV P...,NaN,Glaxo Smith Kline,Unknown,NaN,2001-08-31,HIV Infections,Drug: Abacavir sulfate\nDrug: Nelfinavir mesyl...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,['HIV-1\r\n \t \t Dru...,['HIV Infections Bl...,NaN,Drug,210.0,"DB00220, DB01048, DB01319"
424052,Effectiveness and Safety of Two Forms of Stavu...,NaN,Bristol-Myers Squibb,Completed,2001-08-31,2011-05-04,HIV Infections,Drug: Efavirenz\nDrug: Lamivudine\nDrug: Stavu...,Phase 3,Interventional (Clinical Trial),...,NaN,NaN,NaN,Empty,"['Drug Therapy, Combination\r\n \t ...",['HIV Infections Bl...,3533 days,Drug,730.0,DB00649


In [445]:
# Get and print unique values from the specified column
unique_values = top20Companies["sponsor"].unique()
print(unique_values)
print(len(unique_values))
print(len(top20Companies))

['AstraZeneca' 'Johnson & Johnson' 'Hoffmann-La Roche' 'Glaxo Smith Kline'
 'Bayer' 'Pfizer' 'Novartis' 'AbbVie' 'Amgen' 'Sanofi'
 'Bristol-Myers Squibb' 'Eli Lilly' 'Gilead' 'Genentech' 'Novo Nordisk'
 'Astellas' 'Boehringer Ingelheim' 'Merck' 'Eisai' 'Takeda' 'Celgene']
21
23882


In [446]:
# Split the 'name' column by comma and stack the DataFrame
df_split = top20Companies.assign(name=top20Companies['DrugBank_ID'].str.split(', ')).explode('DrugBank_ID')

# Filter DataFrame based on IDs from the list
filtered_df = df_split[df_split['DrugBank_ID'].isin(id_list)]

# Group by 'ID' and 'name' in the filtered DataFrame, count occurrences, then pivot the table
grouped_df = filtered_df.groupby(['sponsor', 'DrugBank_ID']).size().reset_index(name='count')
pivot_df_company = grouped_df.pivot(index='sponsor', columns='DrugBank_ID', values='count').fillna(0)

pivot_df_company = pivot_df_company.T

pivot_df_company['DrugBank_Approval'] = None
pivot_df_company['Nature_Approvals'] = None

for index,row in pivot_df_company.iterrows():
    for index2, row2 in df.iterrows():
        if index == row2["DrugBank ID"]:
            pivot_df_company.loc[index, 'DrugBank_Approval'] = row2["DB_Approval"]
            pivot_df_company.loc[index, 'Nature_Approvals'] = row2["Nature_Approvals"]

In [447]:
pivot_df_company

sponsor,AbbVie,Amgen,Astellas,AstraZeneca,Bayer,Boehringer Ingelheim,Bristol-Myers Squibb,Celgene,Eisai,Eli Lilly,...,Hoffmann-La Roche,Johnson & Johnson,Merck,Novartis,Novo Nordisk,Pfizer,Sanofi,Takeda,DrugBank_Approval,Nature_Approvals
DrugBank_ID,,,,,,,,,,,,,,,,,,,,,
DB00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Approved,None
DB00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Approved, Investigational",None
DB00005,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,"Approved, Investigational",NA
DB00007,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,"Approved, Investigational",NA
DB00008,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Approved, Investigational",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DB18474,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Investigational,None
DB18549,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Investigational,None
DB18662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Investigational,None


In [448]:
# Filter and print non-None or non-NA values from the specified column
# Replace "NA" values with None in the specified column
pivot_df_company["Nature_Approvals"] = pivot_df_company["Nature_Approvals"].replace('NA', None)
non_missing_values = pivot_df_company[pivot_df_company["Nature_Approvals"].notna()]["Nature_Approvals"].tolist()
print(non_missing_values)
print(len(set(non_missing_values)))

['Collagenase clostridium', 'Hizentra', 'Perjeta', 'IncobotulinumtoxinA', 'Tbo-Filgrastim', '2011', 'Ranexa', '2011', 'Eraxis', 'Benzyl alcohol', '2008', 'Rifamycin', 'Ella', 'Ella', 'Ella', 'Gattex', 'Pylera', '2011', 'Tekturna', 'Kisqali', 'Rifamycin', 'Amitiza', '2011', 'Sprycel', 'Altabax', 'Tykerb', 'Januvia', 'Dacogen', 'Noxafil', 'Prezista', 'Telbivudine', 'Invega', 'Sutent', 'Vectibix', 'Lucentis', 'Chantix', 'Ella', 'Omnaris', 'Afinitor', 'Uridine', 'Daliresp', 'Zolinza', 'Selzentry', 'Ixempra', 'Uloric', 'Multaq', 'Tasigna', 'Lorcaserin', 'Fanapt', 'Esbriet', 'Ezogabine', 'Eucrisa', 'Cotellic', 'Caprelsa', 'Nplate', 'Telaprevir', 'Cyramza', 'Otezla', 'Stelara', 'Zytiga', 'Inotuzumab ozogamicin', 'Cariprazine', 'Lartruvo', 'Cablivi', 'Ilaris', 'Istodax', 'Yervoy', 'Firazyr', 'Rufinamide', 'Nesina', 'Tapentadol', 'Bridion', 'Effient', 'Eltrombopag', 'Doripenem', 'Regadenoson', 'Dalvance', 'Xarelto', 'Livtencity', 'Actemra', 'Torisel', 'Simeprevir', 'Farxiga', 'Empliciti', 'Ongl

In [449]:
# Count rows where 'DrugBank_Approval' includes 'Approved' and 'Nature_Approvals' is None
count_rows = len(pivot_df_company[(pivot_df_company['DrugBank_Approval'].str.contains('Approved')) & (pivot_df_company['Nature_Approvals'].isnull())])

print("Number of rows where 'DB' includes 'Approved' and 'Nature_Approvals' is None:", count_rows)

Number of rows where 'DB' includes 'Approved' and 'Nature_Approvals' is None: 519


In [450]:
1739-519

1220

In [452]:
289/1220*100

23.688524590163933

In [389]:
df.columns

Index(['DrugBank ID', 'Accession Numbers', 'Common name', 'CAS', 'UNII',
       'Synonyms', 'Standard InChI Key', 'Brand_Names', 'DB_Approval',
       'External_IDs'],
      dtype='object')

In [ ]:
# Merge only the 'Names' column from df2 to df1 based on 'ID'
merged_df = pd.merge(df1, df2[['ID', 'Names']], on='ID', how='left')

print(merged_df)

In [372]:
import pandas as pd

# Sample DataFrames (replace with your actual DataFrames)
data1 = {'ID': [1, 2, 3],
         'Value1': [10, 20, 30]}
data2 = {'ID': [1, 2],
         'Names': ['Alice', 'Bob']}
df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2)

# Merge only the 'Names' column from df2 to df1 based on 'ID'
merged_df = pd.merge(df1, df2[['ID', 'Names']], on='ID', how='left')

print(merged_df)


   ID  Value1  Names
0   1      10  Alice
1   2      20    Bob
2   3      30    NaN


In [418]:
%%time
top20Companies['DB_Approval'] = ""
top20Companies['Nature_Approvals'] = ""

# Iterate over df1 and update Name and Age based on matching IDs in df2
for index, row in top20Companies.iterrows():
    id_value = row['DrugBank_ID']
    match_row = df[df['DrugBank ID'] == id_value]
    if not match_row.empty:
        top20Companies.at[index, 'Drug_Bank_Approval'] = str(match_row['DB_Approval'].values[0])
        top20Companies.at[index, 'Nature_Approval'] = str(match_row['Nature_Approvals'].values[0])
    else:
        top20Companies.at[index, 'Drug_Bank_Approval'] = None
        top20Companies.at[index, 'Nature_Approval'] = None

CPU times: user 28 s, sys: 210 ms, total: 28.2 s
Wall time: 29 s


In [344]:
top20Companies

,titles,Responsible_Party,sponsor,RecruitmentStatus,FirstPosted,LastUpdatePosted,condition,treatment,phase,StudyType,...,Keywords,AdditonalRelevantPoints,Post_updated_for,category,participants,DrugBank_ID,DB_Approval,Nature_Approvals,Drug_Bank_Approval,Nature_Approval
56,"A Study to Investigate the Safety, Tolerabilit...",AstraZeneca,AstraZeneca,Recruiting,2022-12-22,2022-12-22,Inflammatory Diseases,Drug: AZD673\nDrug: Placebo,Phase 1,Interventional (Clinical Trial),...,['IRAK4 kinase inhibitor\r\n \t ...,Empty,0 days,Drug,133.0,None,None,None,NaN,NaN
70,Clinical Evaluation of a Multidose Preservativ...,"Johnson & Johnson Vision Care, Inc.","Johnson &amp; Johnson Vision Care, Inc.",Notyetrecruiting,2022-12-22,2022-12-22,Dry Eye,Drug: Investigational Lubricating Eye Drop in ...,Phase 2,Interventional (Clinical Trial),...,Empty,['Dry Eye Syndromes ...,0 days,Drug,116.0,None,None,None,NaN,NaN
92,A Study to Evaluate the Safety and Pharmacokin...,Hoffmann-La Roche,Hoffmann-La Roche,Notyetrecruiting,2022-12-22,2022-12-22,PD-L1-selected Solid Tumors,Drug: Tiragolumab and Atezolizumab IV FDC,Phase 2,Interventional (Clinical Trial),...,Empty,['Neoplasms Atezoli...,0 days,Drug,60.0,"DB11595, DB16409",None,None,NaN,NaN
194,"A Study to Investigate the Safety, Tolerabilit...",GlaxoSmithKline,GlaxoSmithKline,Notyetrecruiting,2022-12-21,2022-12-21,"Hematologic Diseases\nAnaemia, Sickle Cell",Drug: GSK4172239D\nOther: Placebo,Phase 1,Interventional (Clinical Trial),...,['Hematologic diseases\r\n \t ...,"['Anemia, Sickle Cell ...",0 days,Drug,40.0,None,None,None,NaN,NaN
447,A Study to Learn How Well the Study Treatment ...,Bayer,Bayer,Notyetrecruiting,2022-12-19,2022-12-19,Atopic Dermatitis,Drug: Zabedosertib (BAY1834845)\nDrug: Placebo...,Phase 2,Interventional (Clinical Trial),...,Empty,"['Dermatitis, Atopic ...",0 days,Drug,72.0,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423975,"A Comparison of BMS-232632 With Efavirenz, Eac...",NaN,Bristol-Myers Squibb,Completed,2001-08-31,2011-05-04,HIV Infections,Drug: Atazanavir\nDrug: Lamivudine/Zidovudine\...,Phase 3,Interventional (Clinical Trial),...,['Placebos\r\n \t \t ...,['HIV Infections Bl...,3533 days,Drug,NaN,DB01072,None,None,"Approved, Investigational",NA
423990,Comparison of GW433908/Ritonavir to Nelfinavir...,NaN,Glaxo Wellcome,Unknown,NaN,2001-08-31,HIV Infections,Drug: Ritonavir\nDrug: Abacavir sulfate\nDrug:...,Phase 3,Interventional (Clinical Trial),...,['HIV-1\r\n \t \t Dru...,['HIV Infections Bl...,NaN,Drug,624.0,"DB00220, DB00503, DB00709, DB01048",None,None,NaN,NaN
423995,Comparison of GW433908 and Nelfinavir in HIV P...,NaN,Glaxo Wellcome,Unknown,NaN,2001-08-31,HIV Infections,Drug: Abacavir sulfate\nDrug: Nelfinavir mesyl...,Phase 3,Interventional (Clinical Trial),...,['HIV-1\r\n \t \t Dru...,['HIV Infections Bl...,NaN,Drug,210.0,"DB00220, DB01048, DB01319",None,None,NaN,NaN
424052,Effectiveness and Safety of Two Forms of Stavu...,NaN,Bristol-Myers Squibb,Completed,2001-08-31,2011-05-04,HIV Infections,Drug: Efavirenz\nDrug: Lamivudine\nDrug: Stavu...,Phase 3,Interventional (Clinical Trial),...,"['Drug Therapy, Combination\r\n \t ...",['HIV Infections Bl...,3533 days,Drug,730.0,DB00649,None,None,"Approved, Investigational",NA


In [292]:
trialed_drugs_df["Nature_Approval"] = ""
for index, row in trialed_drugs_df.iterrows():
    for i, r in smaller_approval_df.iterrows():
        if row["Common name"].strip().lower() in str(r["Generic name (Trade name)"]).lower():
            #print(r['year'])
            trialed_drugs_df.loc[index, 'DrugBank_ID'] = r['year']
        else:
            trialed_drugs_df.loc[index, 'DrugBank_ID'] = None

In [298]:
trialed_drugs_df.drop(columns=['DrugBank_ID'])

,DrugBank ID,Common name,Synonyms,Nature_Approval
1,DB00002,Cetuximab,Cetuximab | Cétuximab | Cetuximabum,
3,DB00004,Denileukin diftitox,Denileukin | Denileukin diftitox | Interleukin...,
4,DB00005,Etanercept,Etanercept | etanercept-szzs | etanercept-ykro...,
6,DB00007,Leuprolide,Leuprorelin | Leuprorelina | Leuproreline | Le...,
7,DB00008,Peginterferon alfa-2a,PEG-IFN alfa-2A | PEG-Interferon alfa-2A | Peg...,
...,...,...,...,...
16485,DB18622,ABBV-011,"an antibody-drug conjugate of humanized, cyste...",
16548,DB18685,Whey,"Allergenic Extract- Whey, Cow's Milk | Bovine ...",
16558,DB18695,Metelimumab,Human anti-transforming growth factor beta 1 m...,
16572,DB18709,Ropsacitinib,"Cyclobutaneacetonitrile, 3-cyano-1-(4-(6-(1-me...",


In [ ]:
%%time
# scrape for
from selenium import webdriver
from selenium.webdriver.common.by import By

trialed_drugs_df["DB_Approval"] = ""
# Set up the Chrome driver
driver = webdriver.Chrome()
for index, row in trialed_drugs_df.iterrows():
    # URL of the website
    url = f"https://go.drugbank.com/drugs/{row["DrugBank ID"]}"
    print(row["DrugBank ID"])

    # Open the URL
    driver.get(url)
    try:
        # Check if the element with ID 'external-ids' is present
        external_ids_element = driver.find_element(By.ID, 'groups')

        if external_ids_element:
            # If the element is present, find the next <dd> element and print its text
            next_dd_element = external_ids_element.find_element(By.XPATH, "./following-sibling::dd")
            #print(next_dd_element.text)
            value = next_dd_element.text
            #print(value)
            trialed_drugs_df.loc[index, 'DB_Approval'] = value
        else:
            print(f"Element with ID 'external-ids' found but Error at {row["DrugBank ID"]}")
            #external_IDs.loc[len(external_IDs)] = None
    except:
        #print(f"Element with ID 'external-ids' not found at {ID}")
        #external_IDs.loc[len(external_IDs)] = None
        print("other error")


In [301]:
trialed_drugs_df

,DrugBank ID,Common name,Synonyms,Nature_Approval,DrugBank_ID,DB_Approval
1,DB00002,Cetuximab,Cetuximab | Cétuximab | Cetuximabum,,Approved,
3,DB00004,Denileukin diftitox,Denileukin | Denileukin diftitox | Interleukin...,,"Approved, Investigational",
4,DB00005,Etanercept,Etanercept | etanercept-szzs | etanercept-ykro...,,"Approved, Investigational",
6,DB00007,Leuprolide,Leuprorelin | Leuprorelina | Leuproreline | Le...,,"Approved, Investigational",
7,DB00008,Peginterferon alfa-2a,PEG-IFN alfa-2A | PEG-Interferon alfa-2A | Peg...,,"Approved, Investigational",
...,...,...,...,...,...,...
16485,DB18622,ABBV-011,"an antibody-drug conjugate of humanized, cyste...",,Investigational,
16548,DB18685,Whey,"Allergenic Extract- Whey, Cow's Milk | Bovine ...",,Investigational,
16558,DB18695,Metelimumab,Human anti-transforming growth factor beta 1 m...,,Investigational,
16572,DB18709,Ropsacitinib,"Cyclobutaneacetonitrile, 3-cyano-1-(4-(6-(1-me...",,Investigational,


In [ ]:
# Transfrom the string into a list or the entire thing into a dictonary 

In [ ]:
# Iterrate over the big dataframe and check if any value from the dictonary is in the most important columns. 
    # If a match is found a) add a new column with the Drugbank ID and b) an new column with the match
    # Print out the time of the ordeal 

In [ ]:
# Create a subset for Pharma sponsored trials

In [ ]:
# Compute PoS for both (use drugs@FDA and nature values)

In [ ]:
# Show differences in PoS